In [1]:
import os
from enum import Enum
import gzip
import time

import numpy as np
from scipy.sparse import dok_matrix, csr_matrix
import tensorflow as tf

# Attalos Imports
import attalos.util.log.log as l
from attalos.dataset.dataset import Dataset
from attalos.evaluation.evaluation import Evaluation

# Local models
from mse import MSEModel
from negsampling import NegSamplingModel
from fast0tag import FastZeroTagModel

In [2]:
# Setup global objects
logger = l.getLogger(__name__)

In [3]:
from attalos.imgtxt_algorithms.regress2sum.multihot import MultihotModel
from attalos.imgtxt_algorithms.regress2sum.naivesum import NaiveSumModel
from attalos.imgtxt_algorithms.regress2sum.wdv import WDVModel
from attalos.dataset.wordvectors.glove import GloveWrapper

In [4]:
# Temp object using duck typing to replace command line arguments
args = lambda: None
#args.image_feature_file_train = "/local_data/teams/attalos/features/image/espgame_train_20160823_inception.hdf5"
#args.text_feature_file_train = "/local_data/teams/attalos/features/text/espgame_train_20160823_text.json.gz"
#args.image_feature_file_test = "/local_data/teams/attalos/features/image/espgame_test_20160823_inception.hdf5"
#args.text_feature_file_test = "/local_data/teams/attalos/features/text/espgame_test_20160823_text.json.gz"
args.image_feature_file_train = "/local_data/teams/attalos/features/image/iaprtc_train_20160816_inception.hdf5"
args.text_feature_file_train = "/local_data/teams/attalos/features/text/iaprtc_train_20160816_text.json.gz"
args.image_feature_file_test = "/local_data/teams/attalos/features/image/iaprtc_test_20160816_inception.hdf5"
args.text_feature_file_test = "/local_data/teams/attalos/features/text/iaprtc_test_20160816_text.json.gz"
args.word_vector_file = "/local_data/kylez/glove.6B.200d.txt"
args.word_vector_type = "glove"
args.model_type = "wdv"
args.cross_eval = False
args.in_memory = True
args.model_input_path = None
args.model_output_path = None
args.num_epochs = 400
args.batch_size = 100
args.learning_rate = 0.0001

In [5]:
class WordVectorTypes(Enum):
    w2v = 1
    glove = 2

In [6]:
class ModelTypes(Enum):
    mse = 1
    negsampling = 2
    fast0tag = 3
    multihot = MultihotModel
    naivesum = NaiveSumModel
    wdv = WDVModel

In [7]:
def train_batch(sess, model, batch):
    train_x, train_y = batch
    training_loss = model.fit(sess, train_x, train_y)
    return training_loss

In [8]:
def train_epoch(sess, model, train_dataset, batch_size):
    training_losses = []
    for cur_batch_num, batch in enumerate(model.to_batches(train_dataset, batch_size)):
        training_loss = train_batch(sess, model, batch)
        training_losses.append(training_loss)
    avg_training_loss = sum(training_losses) / float(len(training_losses))
    return avg_training_loss

In [9]:
def train(sess, model, num_epochs, train_dataset, batch_size, epoch_verbosity_rate=10):
    for cur_epoch in xrange(num_epochs):
        verbose = cur_epoch % epoch_verbosity_rate == 0
        avg_training_loss = train_epoch(sess, model, train_dataset, batch_size)
        if verbose:
            logger.info("Finished epoch %s. (Avg. training loss: %s)" % (cur_epoch, avg_training_loss))

In [10]:
def load_wv_model(word_vector_file, word_vector_type):
    if args.word_vector_type == WordVectorTypes.glove.name:
        from glove import Glove
        glove_model = Glove.load_stanford(word_vector_file)
        wv_model = GloveWrapper(glove_model)
    else: #args.word_vector_type == WordVectorTypes.w2v.name:
        import word2vec
        w2v_model = word2vec.load(word_vector_file)
        wv_model = W2VWrapper(w2v_model)
    return wv_model

In [11]:
logger.info("Parsing train and test datasets.")
train_dataset = Dataset(args.image_feature_file_train, args.text_feature_file_train, load_image_feats_in_mem=args.in_memory)
test_dataset = Dataset(args.image_feature_file_test, args.text_feature_file_test)

logger.info("Reading word vectors from file.")
wv_model = load_wv_model(args.word_vector_file, args.word_vector_type)

[2016-09-09 19:37:07,239] [INFO] Parsing train and test datasets.
[2016-09-09 19:37:07,557] [INFO] Reading word vectors from file.


In [13]:
#sess.close()
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)

In [14]:
model_cls = ModelTypes[args.model_type].value
logger.info("Selecting model class: %s" % model_cls.__name__)
#datasets = [train_dataset] if args.cross_eval else [train_dataset, test_dataset]
model = model_cls(wv_model, train_dataset=train_dataset, test_dataset=test_dataset, **vars(args))
model.initialize_model(sess)

[2016-09-09 19:42:14,013] [INFO] Selecting model class: WDVModel
[2016-09-09 19:42:18,153] [DEBUG] Constructing w1.
[2016-09-09 19:42:25,909] [DEBUG] No specified vocab2. Setting w2 = w1.
[2016-09-09 19:42:25,912] [DEBUG] Multiplying w1 and w2.
[2016-09-09 19:42:25,916] [DEBUG] Applying preprocess_fn.
[2016-09-09 19:42:26,021] [INFO] Input size: 2048
[2016-09-09 19:42:26,022] [INFO] Output size: 288
[2016-09-09 19:42:26,025] [INFO] Hidden layer size: 1168
[2016-09-09 19:42:26,041] [INFO] Hidden layer size: 1168
[2016-09-09 19:42:26,069] [INFO] Hidden layer size: 1168


In [15]:
logger.info("Starting training phase.")
train(sess, model, args.num_epochs, train_dataset, args.batch_size) #, train_dataset, wv_model, test_dataset=test_dataset, epoch_verbosity_rate=100)

[2016-09-09 19:42:27,778] [INFO] Starting training phase.
[2016-09-09 19:43:00,889] [INFO] Finished epoch 0. (Avg. training loss: 148.447199865)
[2016-09-09 19:43:47,840] [INFO] Finished epoch 10. (Avg. training loss: 84.4973350005)
[2016-09-09 19:44:12,841] [INFO] Finished epoch 20. (Avg. training loss: 77.1428703395)
[2016-09-09 19:44:37,677] [INFO] Finished epoch 30. (Avg. training loss: 70.1581770724)
[2016-09-09 19:45:02,589] [INFO] Finished epoch 40. (Avg. training loss: 63.6083343679)
[2016-09-09 19:45:27,424] [INFO] Finished epoch 50. (Avg. training loss: 57.3654346466)
[2016-09-09 19:45:52,148] [INFO] Finished epoch 60. (Avg. training loss: 51.9774155834)
[2016-09-09 19:46:16,772] [INFO] Finished epoch 70. (Avg. training loss: 47.4211089394)
[2016-09-09 19:46:41,418] [INFO] Finished epoch 80. (Avg. training loss: 43.7574630434)
[2016-09-09 19:47:05,984] [INFO] Finished epoch 90. (Avg. training loss: 40.6061665362)
[2016-09-09 19:47:30,622] [INFO] Finished epoch 100. (Avg. trai

KeyboardInterrupt: 

In [16]:
logger.info("Starting evaluation phase.")
test_x, test_y = model.to_ndarrs(test_dataset)
predictions = model.predict(sess, test_x)
evaluator = Evaluation(test_y, predictions, k=5)
logger.info("Evaluation (precision, recall, f1): %s" % evaluator.evaluate())

[2016-09-09 19:54:02,409] [INFO] Starting evaluation phase.
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[2016-09-09 19:54:02,965] [INFO] Evaluation (precision, recall, f1): [0.40860336819079413, 0.28596257579458823, 0.31335328330840467]


In [24]:
predictions = sess.run([model.model_info["predictions"]], feed_dict={
        model.model_info["input"]: test_x
    })

In [14]:
def convert_args_and_call_model(args):
    #logger.info("Parsing train and test datasets.")
    #train_dataset = Dataset(args.image_feature_file_train, args.text_feature_file_train, load_image_feats_in_mem=args.in_memory)
    #test_dataset = Dataset(args.image_feature_file_test, args.text_feature_file_test)
    
    #logger.info("Reading word vectors from file.")
    #wv_model = load_wv_model(args.word_vector_file, args.word_vector_type)
    
    #model_cls = ModelTypes[args.model_type].value
    #logger.info("Selecting model class: %s" % model_cls.__name__)
    #datasets = [train_dataset] if args.cross_eval else [train_dataset, test_dataset]
    #model = model_cls(wv_model, train_dataset=train_dataset, test_dataset=test_dataset, **vars(args))
    
    config = tf.ConfigProto(log_device_placement=True)
    config.gpu_options.allow_growth = True
    with tf.Session(config=config) as sess:
        model.initialize_model(sess)
        if args.model_input_path:
            model.load(sess, args.model_input_path)
        logger.info("Starting training phase.")
        train(sess, model, args.num_epochs, train_dataset, args.batch_size) #, train_dataset, wv_model, test_dataset=test_dataset, epoch_verbosity_rate=100)
        if args.model_output_path:
            model.save(sess, args.model_output_path)
        
        logger.info("Starting evaluation phase.")
        test_x, test_y = model.to_ndarrs(test_dataset)
        predictions = model.predict(sess, test_x)
        evaluator = Evaluation(test_y, predictions, k=5)
        logger.info("Evaluation (precision, recall, f1): %s" % evaluator.evaluate())

In [15]:
convert_args_and_call_model(args)

[2016-09-02 20:13:04,838] [INFO] Starting training phase.
[2016-09-02 20:13:07,664] [INFO] Finished epoch 0. (Avg. training loss: 21181.0328813)
[2016-09-02 20:13:33,584] [INFO] Finished epoch 10. (Avg. training loss: 12184.0612016)
[2016-09-02 20:13:59,381] [INFO] Finished epoch 20. (Avg. training loss: 8148.95130782)
[2016-09-02 20:14:25,179] [INFO] Finished epoch 30. (Avg. training loss: 6100.9630876)
[2016-09-02 20:14:50,940] [INFO] Finished epoch 40. (Avg. training loss: 4878.23501587)
[2016-09-02 20:15:16,740] [INFO] Finished epoch 50. (Avg. training loss: 4277.74039112)
[2016-09-02 20:15:42,596] [INFO] Finished epoch 60. (Avg. training loss: 3689.44655817)
[2016-09-02 20:16:08,485] [INFO] Finished epoch 70. (Avg. training loss: 3387.55152754)
[2016-09-02 20:16:34,360] [INFO] Finished epoch 80. (Avg. training loss: 3101.4677665)
[2016-09-02 20:17:00,272] [INFO] Finished epoch 90. (Avg. training loss: 2924.60223944)
[2016-09-02 20:17:26,085] [INFO] Finished epoch 100. (Avg. traini

KeyboardInterrupt: 

In [14]:
def main():
    import argparse

    parser = argparse.ArgumentParser(description='Two layer linear regression')
    parser.add_argument("image_feature_file_train",
                        type=str,
                        help="Image Feature file for the training set")
    parser.add_argument("text_feature_file_train",
                        type=str,
                        help="Text Feature file for the training set")
    parser.add_argument("image_feature_file_test",
                        type=str,
                        help="Image Feature file for the test set")
    parser.add_argument("text_feature_file_test",
                        type=str,
                        help="Text Feature file for the test set")
    parser.add_argument("word_vector_file",
                        type=str,
                        help="Text file containing the word vectors")

    # Optional Args
    parser.add_argument("--learning_rate",
                        type=float,
                        default=.001,
                        help="Learning Rate")
    parser.add_argument("--epochs",
                        type=int,
                        default=200,
                        help="Number of epochs to run for")
    parser.add_argument("--batch_size",
                        type=int,
                        default=128,
                        help="Batch size to use for training")
    parser.add_argument("--network",
                        type=str,
                        default="200,200",
                        help="Define a neural network as comma separated layer sizes")
    parser.add_argument("--model_type",
                        type=str,
                        default="mse",
                        choices=['mse', 'negsampling', 'fast0tag'],
                        help="Loss function to use for training")
    parser.add_argument("--in_memory",
                        action='store_true',
                        default="store_false",
                        help="Load training image features into memory for faster training")
    parser.add_argument("--model_input_path",
                        type=str,
                        default=None,
                        help="Model input path (to continue training)")
    parser.add_argument("--model_output_path",
                        type=str,
                        default=None,
                        help="Model output path (to save training)")
    
    # new args
    parser.add_argument("--cross_eval",
                        action="store_true",
                        default=False)
    parser.add_argument("--word_vector_type",
                       type=str,
                       choices=[t.name for t in WordVectorType],
                       help="Format of word_vector_file")

    args = parser.parse_args()

    try:
        # Sacred Imports
        from sacred import Experiment
        from sacred.observers import MongoObserver

        from sacred.initialize import Scaffold

        # Monkey patch to avoid having to declare all our variables
        def noop(item):
            pass
        Scaffold._warn_about_suspicious_changes = noop

        ex = Experiment('Regress2sum')
        ex.observers.append(MongoObserver.create(url=os.environ['MONGO_DB_URI'],
                                             db_name='attalos_experiment'))
        ex.main(lambda: convert_args_and_call_model(args))
        ex.run(config_updates=args.__dict__)
    except ImportError:
        # We don't have sacred, just run the script
        convert_args_and_call_model(args)


if __name__ == '__main__':
    main()

NameError: global name 'WordVectorType' is not defined

In [ ]:
def evaluate_regressor(sess, model, val_image_feats, val_one_hot, wordmatrix, k=5, verbose=False):
    """
    Takes a regressor and returns the precision/recall on the test data
    Args:
        sess: A tensorflow session
        model_info: A dictionary containing tensorflow layers (specifically input and prediction)
        val_image_feats: Image features to test performance on
        val_text_tags: Text Tags to test performance on
        w2v_model: a dictionary like object where the keys are words and the values are word vectors
        k: Top number of items to retrieve to test precision/recall on
        verbose: Verbose output or not

    Returns:
        evaluator: A attalos.evaluation.evaluation.Evaluation object
    """
    val_pred = model.predict(sess, val_image_feats)
    predictions = np.dot(val_pred, wordmatrix.T)

    evaluator = Evaluation(val_one_hot, predictions, k)

    return evaluator

In [ ]:
def create_wordmatrix(w2v_model, dataset=None):
    """
    Take a w2v dictionary and return matrix/index lookup
    Args:
        w2vmodel: Dictionary where keys are words and values are word vectors
        dataset: If specified limits tags in matrix to tags in dataset

    Returns:
        w2ind: Mapping of word to index
        wordmatrix: Numpy matrix of word vectors
    """
    dataset_tags = None
    if dataset:
        dataset_tags = set()
        for tags in dataset.text_feats.values():
            dataset_tags.update(tags)
        num_tags_in_output = len(dataset_tags.intersection(w2v_model.keys()))
    else:
        num_tags_in_output = len(w2v_model)

    # Create word vector matrix to allow for embedding lookup
    w2ind = {}
    wordmatrix = np.zeros((num_tags_in_output, len(w2v_model[w2v_model.keys()[0]])), dtype=np.float32)
    i =0
    for word in w2v_model:
        if dataset_tags is None or word in dataset_tags:
            w2ind[word] = i
            wordmatrix[i, :] = w2v_model[word]
            i += 1
    return w2ind, wordmatrix

In [ ]:
def dataset_to_onehot(dataset, w2ind):
    """
    Take a dataset and prepare it for convient evaluation
    Args:
        dataset: attalos.dataset.dataset object
        w2ind: a dictionary like object mapping words to their index

    Returns:
        img_feats: A matrix of image feautres
        one_hot: A sparse matrix of one hot tags

    """
    image_feat, tags = dataset.get_index(0)

    image_feats = np.zeros((dataset.num_images, image_feat.shape[0]))
    one_hot = dok_matrix((dataset.num_images, len(w2ind)), dtype=np.int32)
    # Extract features and place in numpy matrix
    for i in dataset:
        image_feat, tags = dataset[i]
        image_feats[i, :] = image_feat
        for tag in tags:
            if tag in w2ind:
                one_hot[i, w2ind[tag]] = 1

    return image_feats, csr_matrix(one_hot)

In [ ]:
def train_model(train_dataset,
                test_dataset,
                w2v_model,
                batch_size=128,
                num_epochs=200,
                learning_rate=1.001,
                network_size=[200,200],
                model_input_path = None,
                model_output_path = None,
                verbose=True,
                model_type=ModelTypes.negsampling):
    """
    Train a regression model to map image features into the word vector space
    Args:
        train_dataset: Training attalos.dataset.dataset object
        test_dataset: Test attalos.dataset.dataset object
        w2v_model: A dictionary like object where the keys are words and the values are word vectors
        batch_size: Batch size to use for training
        num_epochs: Number of epochs to train for
        learning_rate: The learning rate for the network
        network_size: A list defining the size of each layer of the neural network
        model_input_path: Path to a file containing initial weights
        model_output_path: Path to save final weights
        verbose: Amounto fdebug information to output
    Returns:
    """
    # Get validation data
    #  Extract features from first image
    image_feats, tags = test_dataset.get_index(0)
    # Get shape and initialize numpy matrix
    image_feat_size = image_feats.shape[0]


    # Turn w2v dictionary into a matrix
    w2ind, word_matrix = create_wordmatrix(w2v_model)
    val_w2ind, val_word_matrix = create_wordmatrix(w2v_model, test_dataset)

    # Precompute onehot representation for evaluation
    val_image_feats, val_one_hot = dataset_to_onehot(test_dataset, val_w2ind)


    # Setup data structures for negative sampling
    if model_type == ModelTypes.negsampling or model_type == ModelTypes.fast0tag:
        word_counts = np.zeros(word_matrix.shape[0])
        for item_id in train_dataset:
            _, tags = train_dataset[item_id]
            for tag in tags:
                if tag in w2ind:
                    word_counts[w2ind[tag]] += 1
        labelpdf = word_counts / word_counts.sum()
        vocabsize = word_matrix.shape[0]
        def negsamp(ignored_inds, num2samp):
            # Negative sampler that takes in indicies

            # Create new probability vector excluding positive samples
            nlabelpdf = np.copy(labelpdf)
            nlabelpdf[ignored_inds] = 0
            nlabelpdf /= nlabelpdf.sum()

            return np.random.choice(vocabsize, size=num2samp, p=nlabelpdf)

    # Time to start building our graph
    with tf.Graph().as_default():
        # Build regressor
        if model_type == ModelTypes.mse:
            logger.info('Building regressor with mean square error loss')
            model = MSEModel(image_feat_size,
                                         word_matrix,
                                        learning_rate=learning_rate,
                                        hidden_units=network_size,
                                        use_batch_norm=True)
        elif model_type == ModelTypes.negsampling:
            logger.info('Building regressor with negative sampling loss')
            model = NegSamplingModel(image_feat_size,
                                        word_matrix,
                                        learning_rate=learning_rate,
                                        hidden_units=network_size,
                                        use_batch_norm=True)
        elif model_type == ModelTypes.fast0tag:
            logger.info('Building model with fast zero tag loss')
            model = FastZeroTagModel(image_feat_size,
                                        word_matrix,
                                        learning_rate=learning_rate,
                                        hidden_units=network_size,
                                        use_batch_norm=True)

        # Allocate GPU memory as needed (vs. allocating all the memory)
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        with tf.Session(config=config) as sess:
            # Initialize model
            model.initialize_model(sess)

            # Optionally restore saved model
            if model_input_path:
                model.load(sess, model_input_path)

            NUM_POSTIVE_EXAMPLES = 5
            NUM_NEGATIVE_EXAMPLES = 10
            # Reuse space for each iteration
            pos_word_ids = np.ones((batch_size, NUM_POSTIVE_EXAMPLES), dtype=np.int32)
            neg_word_ids = np.ones((batch_size, NUM_NEGATIVE_EXAMPLES), dtype=np.int32)
            performance = []
            for epoch in range(num_epochs):
                batch_time_total = 0
                run_time_total = 0

                loss = None
                for batch in range(int(train_dataset.num_images/batch_size)):
                    batch_time = time.time()
                    # Get raw data
                    image_feats, text_tags = train_dataset.get_next_batch(batch_size)

                    # Generate positive examples
                    pos_word_ids.fill(-1)
                    for i, tags in enumerate(text_tags):
                        j = 0
                        for tag in tags:
                            if tag in w2ind and j < NUM_POSTIVE_EXAMPLES:
                                pos_word_ids[i, j] = w2ind[tag]
                                j += 1

                    if model_type == ModelTypes.negsampling or model_type == ModelTypes.fast0tag:
                        neg_word_ids.fill(-1)
                        for i in range(neg_word_ids.shape[0]):
                            neg_word_ids[i] = negsamp(pos_word_ids, NUM_NEGATIVE_EXAMPLES)

                    batch_time = time.time() - batch_time
                    batch_time_total += batch_time

                    run_time = time.time()
                    if model_type == ModelTypes.mse:
                        loss = model.fit(sess, image_feats, pos_word_ids)
                    elif model_type == ModelTypes.negsampling or model_type == ModelTypes.fast0tag:
                        loss = model.fit(sess, image_feats,pos_word_ids, neg_word_ids=neg_word_ids)
                    run_time = time.time() - run_time
                    run_time_total += run_time

                if verbose:
                    eval_time = time.time()
                    evaluator = evaluate_regressor(sess, model, val_image_feats, val_one_hot, val_word_matrix, verbose=False)
                    performance.append(evaluator.evaluate())
                    eval_time = time.time() - eval_time
                    # Evaluate accuracy
                    #print('Epoch {}: Loss: {} Timing: {} {} {}'.format(epoch, loss, batch_time_total, run_time_total, eval_time))
                    logger.debug('Epoch {}: Loss: {} Perf: {} {} {}'.format(epoch, loss, *performance[-1]))

            if model_output_path:
                model.save(sess, model_output_path)

            return performance

In [15]:
print """[2016-09-02 20:28:54,605] [INFO] Starting training phase.
[2016-09-02 20:28:57,225] [INFO] Finished epoch 0. (Avg. training loss: 21108.685902)
[2016-09-02 20:29:23,264] [INFO] Finished epoch 10. (Avg. training loss: 12104.3950084)
[2016-09-02 20:29:49,075] [INFO] Finished epoch 20. (Avg. training loss: 8269.83771584)
[2016-09-02 20:30:15,403] [INFO] Finished epoch 30. (Avg. training loss: 6266.67783425)
[2016-09-02 20:30:41,737] [INFO] Finished epoch 40. (Avg. training loss: 5189.87184837)
[2016-09-02 20:31:07,973] [INFO] Finished epoch 50. (Avg. training loss: 4396.46431108)
[2016-09-02 20:31:34,121] [INFO] Finished epoch 60. (Avg. training loss: 3863.6595348)
[2016-09-02 20:32:00,056] [INFO] Finished epoch 70. (Avg. training loss: 3587.67546914)
[2016-09-02 20:32:25,970] [INFO] Finished epoch 80. (Avg. training loss: 3379.37619296)
[2016-09-02 20:32:52,125] [INFO] Finished epoch 90. (Avg. training loss: 3126.55012651)
[2016-09-02 20:33:17,865] [INFO] Finished epoch 100. (Avg. training loss: 2957.23998885)
[2016-09-02 20:33:43,841] [INFO] Finished epoch 110. (Avg. training loss: 2796.72104298)
[2016-09-02 20:34:09,856] [INFO] Finished epoch 120. (Avg. training loss: 2606.55410073)
[2016-09-02 20:34:35,776] [INFO] Finished epoch 130. (Avg. training loss: 2557.84210968)
[2016-09-02 20:35:01,677] [INFO] Finished epoch 140. (Avg. training loss: 2437.01763084)
[2016-09-02 20:35:27,572] [INFO] Finished epoch 150. (Avg. training loss: 2332.99427587)
[2016-09-02 20:35:53,492] [INFO] Finished epoch 160. (Avg. training loss: 2282.81213587)
[2016-09-02 20:36:19,444] [INFO] Finished epoch 170. (Avg. training loss: 2183.19040819)
[2016-09-02 20:36:45,283] [INFO] Finished epoch 180. (Avg. training loss: 2105.26487801)
[2016-09-02 20:37:11,196] [INFO] Finished epoch 190. (Avg. training loss: 2055.16294514)
[2016-09-02 20:37:37,021] [INFO] Finished epoch 200. (Avg. training loss: 1946.56969729)
[2016-09-02 20:38:02,942] [INFO] Finished epoch 210. (Avg. training loss: 1904.94288358)
[2016-09-02 20:38:28,624] [INFO] Finished epoch 220. (Avg. training loss: 1876.2755016)
[2016-09-02 20:38:54,413] [INFO] Finished epoch 230. (Avg. training loss: 1760.27381689)
[2016-09-02 20:39:20,242] [INFO] Finished epoch 240. (Avg. training loss: 1746.02444597)
[2016-09-02 20:39:46,173] [INFO] Finished epoch 250. (Avg. training loss: 1707.82103105)
[2016-09-02 20:40:12,130] [INFO] Finished epoch 260. (Avg. training loss: 1684.19585557)
[2016-09-02 20:40:37,957] [INFO] Finished epoch 270. (Avg. training loss: 1653.4075685)
[2016-09-02 20:41:03,652] [INFO] Finished epoch 280. (Avg. training loss: 1612.74879039)
[2016-09-02 20:41:29,855] [INFO] Finished epoch 290. (Avg. training loss: 1648.05460843)
[2016-09-02 20:41:55,827] [INFO] Finished epoch 300. (Avg. training loss: 1574.80913405)
[2016-09-02 20:42:21,778] [INFO] Finished epoch 310. (Avg. training loss: 1603.2134122)
[2016-09-02 20:42:47,699] [INFO] Finished epoch 320. (Avg. training loss: 1554.92467221)
[2016-09-02 20:43:13,587] [INFO] Finished epoch 330. (Avg. training loss: 1510.93893363)
[2016-09-02 20:43:39,812] [INFO] Finished epoch 340. (Avg. training loss: 1467.54254913)
[2016-09-02 20:44:05,688] [INFO] Finished epoch 350. (Avg. training loss: 1440.12515883)
[2016-09-02 20:44:31,621] [INFO] Finished epoch 360. (Avg. training loss: 1422.47295414)
[2016-09-02 20:44:57,927] [INFO] Finished epoch 370. (Avg. training loss: 1444.78291529)
[2016-09-02 20:45:24,529] [INFO] Finished epoch 380. (Avg. training loss: 1392.09626389)
[2016-09-02 20:45:50,543] [INFO] Finished epoch 390. (Avg. training loss: 1384.02797976)
[2016-09-02 20:46:16,563] [INFO] Finished epoch 400. (Avg. training loss: 1345.65613417)
[2016-09-02 20:46:42,446] [INFO] Finished epoch 410. (Avg. training loss: 1358.62544875)
[2016-09-02 20:47:08,529] [INFO] Finished epoch 420. (Avg. training loss: 1372.39384114)
[2016-09-02 20:47:34,725] [INFO] Finished epoch 430. (Avg. training loss: 1314.77864317)
[2016-09-02 20:48:00,873] [INFO] Finished epoch 440. (Avg. training loss: 1336.18339539)
[2016-09-02 20:48:27,202] [INFO] Finished epoch 450. (Avg. training loss: 1319.19177315)
[2016-09-02 20:48:53,409] [INFO] Finished epoch 460. (Avg. training loss: 1311.66452096)
[2016-09-02 20:49:19,906] [INFO] Finished epoch 470. (Avg. training loss: 1288.18096993)
[2016-09-02 20:49:46,107] [INFO] Finished epoch 480. (Avg. training loss: 1300.70907697)
[2016-09-02 20:50:12,178] [INFO] Finished epoch 490. (Avg. training loss: 1299.08323045)
[2016-09-02 20:50:38,245] [INFO] Finished epoch 500. (Avg. training loss: 1262.9536653)
[2016-09-02 20:51:04,312] [INFO] Finished epoch 510. (Avg. training loss: 1285.90589003)
[2016-09-02 20:51:30,407] [INFO] Finished epoch 520. (Avg. training loss: 1294.46736214)
[2016-09-02 20:51:56,662] [INFO] Finished epoch 530. (Avg. training loss: 1303.53440371)
[2016-09-02 20:52:22,815] [INFO] Finished epoch 540. (Avg. training loss: 1249.16370045)
[2016-09-02 20:52:48,962] [INFO] Finished epoch 550. (Avg. training loss: 1236.94174333)
[2016-09-02 20:53:15,086] [INFO] Finished epoch 560. (Avg. training loss: 1224.01751952)
[2016-09-02 20:53:41,849] [INFO] Finished epoch 570. (Avg. training loss: 1224.03992566)
[2016-09-02 20:54:08,209] [INFO] Finished epoch 580. (Avg. training loss: 1230.53211316)
[2016-09-02 20:54:34,086] [INFO] Finished epoch 590. (Avg. training loss: 1234.63750805)
[2016-09-02 20:54:59,818] [INFO] Finished epoch 600. (Avg. training loss: 1221.07882378)
[2016-09-02 20:55:26,642] [INFO] Finished epoch 610. (Avg. training loss: 1185.65967248)
[2016-09-02 20:55:52,400] [INFO] Finished epoch 620. (Avg. training loss: 1194.03575446)
[2016-09-02 20:56:18,263] [INFO] Finished epoch 630. (Avg. training loss: 1194.08644208)
[2016-09-02 20:56:44,197] [INFO] Finished epoch 640. (Avg. training loss: 1183.59341639)
[2016-09-02 20:57:10,383] [INFO] Finished epoch 650. (Avg. training loss: 1183.01943727)
[2016-09-02 20:57:36,872] [INFO] Finished epoch 660. (Avg. training loss: 1158.40334736)
[2016-09-02 20:58:02,976] [INFO] Finished epoch 670. (Avg. training loss: 1174.4918095)
[2016-09-02 20:58:29,014] [INFO] Finished epoch 680. (Avg. training loss: 1199.12700064)
[2016-09-02 20:58:54,746] [INFO] Finished epoch 690. (Avg. training loss: 1160.72963125)
[2016-09-02 20:59:20,790] [INFO] Finished epoch 700. (Avg. training loss: 1165.32282188)
[2016-09-02 20:59:46,938] [INFO] Finished epoch 710. (Avg. training loss: 1174.37212615)
[2016-09-02 21:00:13,340] [INFO] Finished epoch 720. (Avg. training loss: 1169.23551282)
[2016-09-02 21:00:39,339] [INFO] Finished epoch 730. (Avg. training loss: 1122.53124688)
[2016-09-02 21:01:05,179] [INFO] Finished epoch 740. (Avg. training loss: 1157.93275036)
[2016-09-02 21:01:31,381] [INFO] Finished epoch 750. (Avg. training loss: 1158.09665056)
[2016-09-02 21:01:57,537] [INFO] Finished epoch 760. (Avg. training loss: 1139.63393333)"""

[2016-09-02 20:28:54,605] [INFO] Starting training phase.
[2016-09-02 20:28:57,225] [INFO] Finished epoch 0. (Avg. training loss: 21108.685902)
[2016-09-02 20:29:23,264] [INFO] Finished epoch 10. (Avg. training loss: 12104.3950084)
[2016-09-02 20:29:49,075] [INFO] Finished epoch 20. (Avg. training loss: 8269.83771584)
[2016-09-02 20:30:15,403] [INFO] Finished epoch 30. (Avg. training loss: 6266.67783425)
[2016-09-02 20:30:41,737] [INFO] Finished epoch 40. (Avg. training loss: 5189.87184837)
[2016-09-02 20:31:07,973] [INFO] Finished epoch 50. (Avg. training loss: 4396.46431108)
[2016-09-02 20:31:34,121] [INFO] Finished epoch 60. (Avg. training loss: 3863.6595348)
[2016-09-02 20:32:00,056] [INFO] Finished epoch 70. (Avg. training loss: 3587.67546914)
[2016-09-02 20:32:25,970] [INFO] Finished epoch 80. (Avg. training loss: 3379.37619296)
[2016-09-02 20:32:52,125] [INFO] Finished epoch 90. (Avg. training loss: 3126.55012651)
[2016-09-02 20:33:17,865] [INFO] Finished epoch 100. (Avg. traini

In [16]:
print """LR = 0.0001
[2016-09-08 19:16:31,476] [INFO] Starting training phase.
[2016-09-08 19:16:59,652] [INFO] Finished epoch 0. (Avg. training loss: 3728.85334084)
[2016-09-08 19:17:41,616] [INFO] Finished epoch 10. (Avg. training loss: 446.906148911)
[2016-09-08 19:18:02,596] [INFO] Finished epoch 20. (Avg. training loss: 230.02659)
[2016-09-08 19:18:23,327] [INFO] Finished epoch 30. (Avg. training loss: 152.64104505)
[2016-09-08 19:18:44,082] [INFO] Finished epoch 40. (Avg. training loss: 114.153651671)
[2016-09-08 19:19:04,807] [INFO] Finished epoch 50. (Avg. training loss: 93.8099693385)
[2016-09-08 19:19:25,501] [INFO] Finished epoch 60. (Avg. training loss: 82.3407104232)
[2016-09-08 19:19:46,054] [INFO] Finished epoch 70. (Avg. training loss: 75.7892449119)
[2016-09-08 19:20:06,673] [INFO] Finished epoch 80. (Avg. training loss: 71.4349764911)
[2016-09-08 19:20:27,154] [INFO] Finished epoch 90. (Avg. training loss: 68.7852852561)
[2016-09-08 19:20:47,797] [INFO] Finished epoch 100. (Avg. training loss: 66.8919837258)
[2016-09-08 19:21:08,646] [INFO] Finished epoch 110. (Avg. training loss: 65.8888210167)
[2016-09-08 19:21:29,276] [INFO] Finished epoch 120. (Avg. training loss: 65.0218931328)
[2016-09-08 19:21:50,186] [INFO] Finished epoch 130. (Avg. training loss: 64.5153313117)
[2016-09-08 19:22:10,671] [INFO] Finished epoch 140. (Avg. training loss: 64.2945017598)
[2016-09-08 19:22:31,350] [INFO] Finished epoch 150. (Avg. training loss: 63.9612216299)
[2016-09-08 19:22:52,175] [INFO] Finished epoch 160. (Avg. training loss: 63.7822946852)
[2016-09-08 19:23:12,963] [INFO] Finished epoch 170. (Avg. training loss: 63.3953738213)
[2016-09-08 19:23:33,933] [INFO] Finished epoch 180. (Avg. training loss: 63.5159262094)
[2016-09-08 19:23:54,722] [INFO] Finished epoch 190. (Avg. training loss: 63.3911959908)
[2016-09-08 19:24:18,345] [INFO] Starting evaluation phase.
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[2016-09-08 19:24:18,893] [INFO] Evaluation (precision, recall, f1): [0.41174798449382544, 0.25697356424325835, 0.28087022940598949]
"""

LR = 0.0001
[2016-09-08 19:16:31,476] [INFO] Starting training phase.
[2016-09-08 19:16:59,652] [INFO] Finished epoch 0. (Avg. training loss: 3728.85334084)
[2016-09-08 19:17:41,616] [INFO] Finished epoch 10. (Avg. training loss: 446.906148911)
[2016-09-08 19:18:02,596] [INFO] Finished epoch 20. (Avg. training loss: 230.02659)
[2016-09-08 19:18:23,327] [INFO] Finished epoch 30. (Avg. training loss: 152.64104505)
[2016-09-08 19:18:44,082] [INFO] Finished epoch 40. (Avg. training loss: 114.153651671)
[2016-09-08 19:19:04,807] [INFO] Finished epoch 50. (Avg. training loss: 93.8099693385)
[2016-09-08 19:19:25,501] [INFO] Finished epoch 60. (Avg. training loss: 82.3407104232)
[2016-09-08 19:19:46,054] [INFO] Finished epoch 70. (Avg. training loss: 75.7892449119)
[2016-09-08 19:20:06,673] [INFO] Finished epoch 80. (Avg. training loss: 71.4349764911)
[2016-09-08 19:20:27,154] [INFO] Finished epoch 90. (Avg. training loss: 68.7852852561)
[2016-09-08 19:20:47,797] [INFO] Finished epoch 100. (Av